# Multi-Agent System KG-Powered

## Overview

AI agent systems: build knowledge graph, implement agent memory, create context graphs, enable multi-agent coordination, and share knowledge.

## Workflow: Build KG → Agent Memory → Context Graphs → Multi-Agent Coordination → Shared Knowledge


In [ ]:
from semantica.ingest import FileIngestor, DBIngestor
from semantica.parse import DocumentParser, StructuredDataParser
from semantica.semantic_extract import NERExtractor, RelationExtractor
from semantica.kg import GraphBuilder
from semantica.context import AgentMemory, ContextRetriever
from semantica.reasoning import InferenceEngine
from datetime import datetime
import os
import tempfile
import json


## Step 1: Build Knowledge Graph


In [ ]:
file_ingestor = FileIngestor()
db_ingestor = DBIngestor()
structured_parser = StructuredDataParser()
ner_extractor = NERExtractor()
relation_extractor = RelationExtractor()
builder = GraphBuilder()

temp_dir = tempfile.mkdtemp()

agents_file = os.path.join(temp_dir, "agents.json")
tasks_file = os.path.join(temp_dir, "tasks.json")

agents_data = [
    {"agent_id": "agent_1", "name": "Research Agent", "role": "researcher"},
    {"agent_id": "agent_2", "name": "Analysis Agent", "role": "analyst"}
]

tasks_data = [
    {"task_id": "task_1", "name": "Data Collection", "status": "completed", "assigned_to": "agent_1"},
    {"task_id": "task_2", "name": "Data Analysis", "status": "in_progress", "assigned_to": "agent_2"}
]

with open(agents_file, 'w') as f:
    json.dump(agents_data, f)
with open(tasks_file, 'w') as f:
    json.dump(tasks_data, f)

file_objects = file_ingestor.ingest_directory(temp_dir, recursive=False)
parsed_agents = structured_parser.parse_json(agents_file)
parsed_tasks = structured_parser.parse_json(tasks_file)

entities = []
relationships = []

for agent_data in parsed_agents.get("data", agents_data):
    entities.append({
        "id": agent_data.get("agent_id", ""),
        "type": "Agent",
        "name": agent_data.get("name", ""),
        "properties": {"role": agent_data.get("role", "")}
    })

for task_data in parsed_tasks.get("data", tasks_data):
    entities.append({
        "id": task_data.get("task_id", ""),
        "type": "Task",
        "name": task_data.get("name", ""),
        "properties": {"status": task_data.get("status", "")}
    })
    
    assigned_agent = task_data.get("assigned_to", "")
    if assigned_agent:
        relationships.append({
            "source": assigned_agent,
            "target": task_data.get("task_id", ""),
            "type": "assigned_to"
        })

knowledge_content = "Market Trends: Analysis shows increasing demand in technology sector."
extracted_entities = ner_extractor.extract(knowledge_content)
extracted_relations = relation_extractor.extract(knowledge_content, extracted_entities)

for entity in extracted_entities[:3]:
    entity_id = f"knowledge_{len([e for e in entities if e['type'] == 'Knowledge']) + 1}"
    entities.append({
        "id": entity_id,
        "type": "Knowledge",
        "name": entity.get("text", entity.get("entity", "")),
        "properties": {}
    })
    
    relationships.append({
        "source": "agent_1",
        "target": entity_id,
        "type": "discovered"
    })
    
    relationships.append({
        "source": "task_1",
        "target": entity_id,
        "type": "produced"
    })

knowledge_graph = builder.build(entities, relationships)

print(f"Ingested {len(file_objects)} files")
print(f"Parsed {len(parsed_agents.get('data', []))} agents and {len(parsed_tasks.get('data', []))} tasks")
print(f"Extracted {len([e for e in entities if e['type'] == 'Knowledge'])} knowledge entities")
print(f"Built knowledge graph with {len(entities)} entities and {len(relationships)} relationships")


## Step 2: Agent Memory


In [ ]:
agent_memory = AgentMemory(knowledge_graph=knowledge_graph)

agent_experiences = [
    {
        "agent_id": "agent_1",
        "content": "Completed data collection task successfully",
        "metadata": {"task": "task_1", "timestamp": datetime.now().isoformat()}
    },
    {
        "agent_id": "agent_2",
        "content": "Started analyzing collected data",
        "metadata": {"task": "task_2", "timestamp": datetime.now().isoformat()}
    }
]

for experience in agent_experiences:
    memory_id = agent_memory.store(
        content=experience["content"],
        metadata=experience["metadata"],
        entities=[{"id": experience["agent_id"], "type": "Agent"}]
    )
    print(f"Stored experience for {experience['agent_id']}: {memory_id}")

print(f"\nTotal memories stored: {agent_memory.stats.get('total_items', 0)}")


## Step 3: Context Graphs


In [ ]:
context_retriever = ContextRetriever(knowledge_graph=knowledge_graph)

def get_agent_context(agent_id, query, kg, retriever):
    context_query = f"{query} for agent {agent_id}"
    retrieved_context = retriever.retrieve(
        query=context_query,
        max_results=5,
        use_graph_expansion=True,
        max_hops=2,
        entity_ids=[agent_id]
    )
    
    context_items = []
    if retrieved_context:
        context_items.append({
            "type": "agent_info",
            "data": {"agent_id": agent_id}
        })
        
        related_tasks = []
        related_knowledge = []
        
        for ctx in retrieved_context:
            for entity in ctx.related_entities:
                if entity.get("type") == "Task":
                    related_tasks.append(entity)
                elif entity.get("type") == "Knowledge":
                    related_knowledge.append(entity)
        
        context_items.append({
            "type": "related_tasks",
            "data": related_tasks
        })
        context_items.append({
            "type": "related_knowledge",
            "data": related_knowledge
        })
    
    return context_items

context_agent_1 = get_agent_context("agent_1", "What should I work on?", knowledge_graph, context_retriever)
print(f"Context for agent_1: {len(context_agent_1)} context items")
for item in context_agent_1:
    print(f"  - {item['type']}: {len(item['data']) if isinstance(item['data'], list) else 1} items")


## Step 4: Multi-Agent Coordination


In [ ]:
shared_knowledge = knowledge_graph

task_1 = "Analyze market trends"
task_2 = "Review analysis results"

agent_1_context = get_agent_context("agent_1", task_1, shared_knowledge, context_retriever)
agent_2_context = get_agent_context("agent_2", task_2, shared_knowledge, context_retriever)

print("Agent 1 Context:")
for item in agent_1_context:
    if isinstance(item['data'], list):
        print(f"  {item['type']}: {[d.get('name', d.get('id')) for d in item['data']]}")
    else:
        print(f"  {item['type']}: {item['data'].get('name', item['data'].get('id'))}")

print("\nAgent 2 Context:")
for item in agent_2_context:
    if isinstance(item['data'], list):
        print(f"  {item['type']}: {[d.get('name', d.get('id')) for d in item['data']]}")
    else:
        print(f"  {item['type']}: {item['data'].get('name', item['data'].get('id'))}")


## Step 5: Shared Knowledge


In [ ]:
inference_engine = InferenceEngine()

inference_engine.add_rule("IF agent performs action ON entity THEN agent action entity")

agent_actions = [
    {"agent": "agent_1", "action": "discovered", "entity": "knowledge_1"},
    {"agent": "agent_2", "action": "analyzed", "entity": "knowledge_1"},
]

for action in agent_actions:
    inference_engine.add_fact(action)

inferred_results = inference_engine.forward_chain()

new_relationships = []
for action in agent_actions:
    new_relationships.append({
        "source": action["agent"],
        "target": action["entity"],
        "type": action["action"],
        "properties": {"timestamp": datetime.now().isoformat(), "inferred": False}
    })

for result in inferred_results:
    if hasattr(result, 'conclusion') and isinstance(result.conclusion, dict):
        if "agent" in result.conclusion and "entity" in result.conclusion:
            new_relationships.append({
                "source": result.conclusion.get("agent", ""),
                "target": result.conclusion.get("entity", ""),
                "type": result.conclusion.get("action", ""),
                "properties": {"timestamp": datetime.now().isoformat(), "inferred": True}
            })

new_knowledge = {
    "entities": [],
    "relationships": new_relationships
}

if new_knowledge["relationships"]:
    updated_kg = builder.build(
        knowledge_graph.get("entities", []) + new_knowledge["entities"],
        knowledge_graph.get("relationships", []) + new_knowledge["relationships"]
    )
    print(f"Updated knowledge graph with {len(new_knowledge['relationships'])} new relationships")
else:
    updated_kg = knowledge_graph

entities_count = len(updated_kg.get("entities", []))
relationships_count = len(updated_kg.get("relationships", []))

print(f"\nShared knowledge graph has {entities_count} entities and {relationships_count} relationships")


## Summary

Multi-agent system workflow:
- Knowledge graph built
- Agent memory implemented
- Context graphs created
- Multi-agent coordination enabled
- Shared knowledge maintained
